In [47]:
import psycopg2

# Database connection parameters: root_user
conn_string = "host='140.112.12.188' user='postgres' dbname='dbms_final' password='1234' port='5432'"

# Connect to the database
conn = psycopg2.connect(conn_string)
print("Connection established")

# Create a cursor object
cursor = conn.cursor()

# Define the SQL for creating the function with logging
create_function_sql = """
CREATE FUNCTION site_policy(row_site int8, curr_user text)
RETURNS bool
AS $$
BEGIN
    RETURN (
        (SELECT Site_ID
        FROM Doctor WHERE UserName = curr_user) 
        = row_site
        );

END;
$$
LANGUAGE plpgsql;
"""
# Define the SQL for enabling row-level security and creating the policy
enable_rls_sql = """
ALTER TABLE patient
ENABLE ROW LEVEL SECURITY;

CREATE POLICY doctor_read
ON Patient FOR SELECT
USING (site_policy(Site_ID, current_user));
"""

# Execute the SQL commands
try:
    # Drop policy if it exists
    cursor.execute("DROP POLICY IF EXISTS doctor_read ON Patient;")
    cursor.execute("DROP FUNCTION IF EXISTS site_policy(int8, text);")
    # Create the function
    cursor.execute(create_function_sql)
    print("Function created successfully")
    # Enable row-level security and create the policy
    cursor.execute(enable_rls_sql)
    conn.commit()
    print("Row-level security enabled and policy created successfully")
    """
    cursor.execute("SELECT relrowsecurity, relforcerowsecurity FROM pg_catalog.pg_class WHERE relname = 'patient';")
    print(cursor.fetchall())
    cursor.execute("SELECT * FROM pg_policies WHERE policyname = 'doctor_read' AND tablename = 'patient';")
    print(cursor.fetchall())
    
    # Set application user ID using custom configuration        
    cursor.execute("SELECT set_config('app.current_user_id', '5', false);")
    
    # Execute the select query
    cursor.execute("SELECT * FROM patient WHERE ID_Number = '9666427225';")
    results = cursor.fetchall()
    print("Query executed successfully, results:")
    print(results)
    """
except Exception as e:
    conn.rollback()
    print(f"An error occurred: {e}")


# Close the cursor and connection
cursor.close()
conn.close()

Connection established
Function created successfully
Row-level security enabled and policy created successfully


In [69]:
import time
# Doctor with site_ID 0
conn_string = "host='140.112.12.188' user='lily814778' dbname='dbms_final' password='1234' port='5432'"

# Connect to the database
conn = psycopg2.connect(conn_string)
#print("Connection established")

# Create a cursor object
cursor = conn.cursor()
Authorized = []
for i in range(100):
    start_time = time.perf_counter_ns()
    cursor.execute("SELECT * FROM Patient WHERE ID_number='3769016772';")
    end_time = time.perf_counter_ns()
    Authorized.append((end_time-start_time)/10**3)
cursor.close()
conn.close()

In [79]:
# Another user with no permission
conn_string = "host='140.112.12.188' user='andychang' dbname='dbms_final' password='1234' port='5432'"

# Connect to the database
conn = psycopg2.connect(conn_string)
#print("Connection established")

# Create a cursor object
cursor = conn.cursor()
Unauthorized = []
for i in range(100):
    start_time = time.perf_counter_ns()
    cursor.execute("SELECT * FROM Patient WHERE ID_number='3769016772';")
    end_time = time.perf_counter_ns()
    Unauthorized.append((end_time-start_time)/10**3)
cursor.close()
conn.close()

In [80]:
# Another user with no permission and ID isn't in dataset
conn_string = "host='140.112.12.188' user='andychang' dbname='dbms_final' password='1234' port='5432'"

# Connect to the database
conn = psycopg2.connect(conn_string)
#print("Connection established")

# Create a cursor object
cursor = conn.cursor()
Non_Existent = []
for i in range(100):
    start_time = time.perf_counter_ns()
    cursor.execute("SELECT * FROM Patient WHERE ID_number='3769016772';")
    end_time = time.perf_counter_ns()
    Non_Existent.append((end_time-start_time)/10**3)
cursor.close()
conn.close()